In [ ]:
import numpy as np
from itertools import combinations

I12 = np.eye(12, dtype=int)
P = np.array([
        [1,1,1,0,1,1,1,0,0,0,1,0],
        [1,0,1,1,0,1,1,1,0,0,0,1],
        [1,1,0,1,1,0,1,1,1,0,0,0],
        [1,0,1,0,1,1,0,1,1,1,0,0],
        [1,0,0,1,0,1,1,0,1,1,1,0],
        [1,0,0,0,1,0,1,1,0,1,1,1],
        [1,1,0,0,0,1,0,1,1,0,1,1],
        [1,1,1,0,0,0,1,0,1,1,0,1],
        [1,1,1,1,0,0,0,1,0,1,1,0],
        [1,0,1,1,1,0,0,0,1,0,1,1],
        [1,1,0,1,1,1,0,0,0,1,0,1],
        [0,1,1,1,1,1,1,1,1,1,1,1]
    ], dtype=int)
G = np.hstack((I12, P))

def encode_golay(data, G):
    return np.dot(data, G) % 2

def decode_golay(received, G):
    n = G.shape[1]
    k = G.shape[0]
    H = np.hstack((P.T, np.eye(12, dtype=int)))
    syndrome = np.dot(received, H.T) % 2

    if not np.any(syndrome):
        return received[:k], received
    corrected = received.copy()
    for num_errors in range(1, 4):
        for error_positions in combinations(range(n), num_errors):
            error_vector = np.zeros(n, dtype=int)
            for pos in error_positions:
                error_vector[pos] = 1
            if np.array_equal(syndrome, np.dot(error_vector, H.T) % 2):
                corrected = (corrected + error_vector) % 2
                return corrected[:k], corrected

    raise ValueError("Greška nije mogla biti ispravljena!")


data = np.array([1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1])
print("Ulazni podaci:", data)
encoded = encode_golay(data, G)
print("Kodirani podaci:", encoded)
corrupted = encoded.copy()
corrupted[5] ^= 1
corrupted[8] ^= 1
corrupted[11] ^= 1
print("Oštećeni podaci:", corrupted)
decoded, corrected = decode_golay(corrupted, G)
print(f"Dekodirani podaci: {decoded}")
print(f"Ispravljeni podaci: {corrected}")